##Jupyter notebook for processing analysis run 2 as presented in Kitson _et al._ 2017

###Downloading raw read data from SRA

The below cell will download the raw Illumina data associated with the study and rename the files as specified in the text file `SraAccList.txt` provided in the `data` directory.

The cell expects modules of the [SRA Toolkit](http://www.ncbi.nlm.nih.gov/Traces/sra/?view=toolkit_doc) (`prefetch` and `fastq-dump`) to be in your path (tested with SRA Toolkit version 2.3.5). `prefetch` downloads data in sra format and places it per default into `~/ncbi/public/sra/`. `fastq-dump` converts the sra formatted data to gzipped fastq.

In [ ]:
%%bash

sra_path="~/ncbi/public/sra"

cd data

for line in $(cat SraAccList_run2.txt)
do 
    name=$(echo -e "$line" | cut -d "," -f 1)
    acc=$(echo -e "$line" | cut -d "," -f 2)
    
    prefetch $acc
    fastq-dump --split-files --gzip --defline-seq '@$ac-$sn/$ri' --defline-qual '+' $sra_path/$acc.sra
    mv $acc\_1.fastq.gz $name\_1.fastq.gz
    mv $acc\_2.fastq.gz $name\_2.fastq.gz
done

cd ..

###Processing read data
To ensure complete reproducibility all subsequent processing of the read data was performed in a docker container which we have made available [here](https://hub.docker.com/r/chrishah/metabeat/).

The following cell will print out the help file for metaBEAT_global.py allowing readers to follow the commands used below.

In [ ]:
%%bash

metaBEAT_global.py -h

###PERFORM A CLUSTERING ANALYSIS TO CHECK FOR OPTIMAL CLUSTER READ DEPTH
We will ultimately be clustering and filtering our data to unique sequences but we don't know what read depth we should accept as a minimum valid cluster size. To investigate this we will iterate across different minimum read depths and examine the effect this has on clusters retained for taxonomic assignment.

In [ ]:
%%bash

# Move to the correct folder
mkdir -p ../Processed_data/Cluster_size
cd ../Processed_data/Cluster_size

# Loop across minimum cluster sizes
for i in $(seq 71 5 101)
do
      echo -e "running with coverage $i"
      metaBEAT_global.py -Q ../../data/QueryMap_all.txt --cluster --merge --merged_only \
      --length_filter 310 --product_length 400 --clust_match 1.0 --clust_cov $i \
      --trim_minlength 100 -@ James.Kitson@newcastle.ac.uk -o metaBEAT &> cluster_log$i.txt
      mv metaBEAT_read_stats.csv metaBEAT_read_stats_$i.csv
done

# Combine the files into one large output for plotting in R
cat metaBEAT_read_stats_6.csv | head -n 1 > ../../data/Richard_combined_read_stats_all.csv
cat metaBEAT_read_stats_* | grep "sample," -v >> ../../data/Richard_combined_read_stats_all.csv
cd ..



###PERFORM A MISTAGGING ANALYSIS TO CHECK FOR BACKGROUND NOISE READ DEPTH
We also need to check the illegal tag combinations to make sure that mistagging is minimal.

In [ ]:
%%bash

# Move to the correct folder
mkdir -p ../Processed_data/Cluster_size/run2_mistag
cd ../Processed_data/Cluster_size/run2_mistag

# Loop across minimum cluster sizes
for i in $(seq 6 5 101)
do
      echo -e "running with coverage $i"
      metaBEAT_global.py -Q ../../../data/Illegal_QueryMap_run2.txt --cluster --merge --merged_only \
      --length_filter 310 --product_length 400 --clust_match 1.0 --clust_cov $i \
      --trim_minlength 100 -@ James.Kitson@newcastle.ac.uk -o metaBEAT &> cluster_log_$i.txt
      mv metaBEAT_read_stats.csv metaBEAT_read_stats_$i.csv
done

# Combine the files into one large output for plotting in R
cat metaBEAT_read_stats_6.csv | head -n 1 > ../../../data/combined_illegal_read_stats_run2.csv
cat metaBEAT_read_stats_* | grep "sample," -v >> ../../../data/combined_illegal_read_stats_run2.csv
cd ..


###PERFORMING FINAL ANALYSES (INCL. TAXONOMIC ASSIGNMENT) FROM RAW READ DATA
####From the clustering analysis we have two possible thresholds for minimum cluster size that we could use:

* Minimum cluster size for clean negatives - __> ~36 reads__.
* Minimum cluster size for stable read depths across wells (i.e. removal of rare sequences) - __> ~51 reads__.

####From the mistagging analysis we have an additional minimum cluster size we can use:

* Minimum cluster size for mistagging removal - __> ~66 reads__.

####We will now iterate across these values and compare sequencing success in the [Rnotebook](https://github.com/HullUni-bioinformatics/Kitson_et_al_NMB/blob/master/R_plotting_notebook_run2.Rmd) to examine what effect this has on the ecological parameters we are interested in:
 
1. Parasitoid identity.
2. Percentage parasitism for each Parasitoid.

In [ ]:
%%bash

# Move to the correct folder
mkdir -p ../Processed_data/Final_analysis
cd ../Processed_data/Final_analysis

metaBEAT_global.py -Q ../../data/QueryMap_all.txt --cluster --merge --merged_only \
      --length_filter 310 --product_length 400 --clust_match 1.0 --clust_cov 50 --trim_minlength 100 -E \
      -n 6 -v --blast --blast_db ../../../../Genbank/nt/nt \
      --min_ident 0.95 --min_ali_length 0.90 -@ James.Kitson@newcastle.ac.uk &> OPM_Richard_BLAST_log.txt
      

MetaBEAT produces files in the .biom format for easy use with QIIME but we will be using the output in R for plotting figures. The next cell strips out all the QIIME .biom formatting and leaves us with a rectangular read table.

In [ ]:
%%bash
cd ../Processed_data/Final_analysis/

cat ./GLOBAL/BLAST_0.95/metaBEAT-by-taxonomy-readcounts.blast.tsv | \
grep "# " -v | sed 's/#//' | sed 's/\.blast//g' | sed 's/ /_/' | perl -ne \
'chomp; @a=split("\t"); pop(@a); $out=join("\t", @a); print "$out\n"' \
> ./GLOBAL/BLAST_0.95/metaBEAT-processed.tsv

The reformatted metaBEAT data could be used as is but we'll transpose our table now to make life easy.

In [ ]:
import pandas as pd


filename = "../Processed_data/Final_analysis/GLOBAL/BLAST_0.95/metaBEAT-processed.tsv"
output_filename = "../data/Richard_metaBEAT_transpose.tsv"

df = pd.read_table(filename)
df = df.transpose()
df.to_csv(output_filename, header=None, sep='\t')